# Instalo dependencias necesarias...

In [1]:
!pip install python-dotenv


In [17]:
!pip install scikit-learn


  Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl (8.3 MB)
  Using cached scipy-1.10.1-cp310-cp310-win_amd64.whl (42.5 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [24]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
                                              0.0/268.0 kB ? eta -:--:--
     -------------                           92.2/268.0 kB 1.7 MB/s eta 0:00:01
     -------------------------------------  266.2/268.0 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 268.0/268.0 kB 2.4 MB/s eta 0:00:00


## Preparo el ambiente

In [40]:
import os
import numpy as np
import nltk

from sklearn.cluster import DBSCAN
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

# Cargar variables de entorno desde el archivo .env
load_dotenv()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dicsys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
# Obtener el valor de la variable de entorno CORPUS_DIR
corpus_dir = os.getenv("CORPUS_DIR")
corpus_dir

'./corpus_dev/'

In [21]:
# Verificar si el directorio existe
if corpus_dir is None or not os.path.isdir(corpus_dir):
    print("El directorio del corpus no es válido.")
    exit()

## Obtenemos el listado de archivos

In [22]:
# Obtener la lista de archivos en el directorio
archivos = os.listdir(corpus_dir)
archivos

['suspicious-document00001.txt',
 'suspicious-document00002.txt',
 'suspicious-document00003.txt',
 'suspicious-document00004.txt',
 'suspicious-document00005.txt']

## Declaramos una funcion que actua como 'segmentador'

In [42]:
def segmentador(contenido):
    return sent_tokenize(contenido)

# Probamos la segmentacion en un solo archivo

In [31]:
# Construir la ruta completa al archivo
ruta_archivo = os.path.join(corpus_dir, archivos[0])

In [43]:
with open(ruta_archivo, 'r', encoding='utf-8') as file:
    contenido = file.read()
    
tokens = segmentador(contenido)
tokens

['\ufeffThe convening of this body gave rise to much dissatisfaction among the people of Texas.',
 'They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.',
 'Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.',
 'The provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling\nof his

## Procesamos el corpus completo

In [44]:
def processAll():
    # Recorrer los archivos del directorio
    for archivo in archivos:
        # Construir la ruta completa al archivo
        ruta_archivo = os.path.join(corpus_dir, archivo)

        # Verificar si es un archivo
        if os.path.isfile(ruta_archivo):
            # Leer el contenido del archivo
            with open(ruta_archivo, 'r', encoding='utf-8') as file:
                contenido = file.read()

            tokens = segmentador(contenido)
            
            # Realizar las operaciones necesarias con el contenido del archivo
            # Aquí puedes agregar tu código para trabajar con el contenido del archivo
            print(f"tokens {archivo}")
            print(tokens)
            print("-" * 50)



In [45]:
processAll()

tokens suspicious-document00001.txt
['\ufeffThe convening of this body gave rise to much dissatisfaction among the people of Texas.', 'They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.', 'Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.', 'The provisional Governor at once rescinded the edict of Governor Murray, 